In [1]:
from solcore import material
from solcore.structure import Layer
import solcore.poisson_drift_diffusion as PDD
import numpy as np

T = 300
wl = np.linspace(350, 1050, 1001) * 1e-9

# First, we create the materials of the QW
QWmat = material('InGaAs')(T=T, In=0.2, strained=True)
Bmat = material('GaAsP')(T=T, P=0.1, strained=True)
i_GaAs = material('GaAs')(T=T)

# The QW is 7 nm wide, with GaAs interlayers 2 nm thick at each side and GaAsP barriers 10 nm thick.
# The final device will have 30 of these QWs.
QW = PDD.QWunit([Layer(width=10e-9, material=Bmat, role="barrier"),
                 Layer(width=2e-9, material=i_GaAs, role="well"),
                 Layer(width=7e-9, material=QWmat, role="well"),
                 Layer(width=2e-9, material=i_GaAs, role="well"),
                 Layer(width=10e-9, material=Bmat, role="barrier")], T=T, repeat=30, substrate=i_GaAs)

# We solve the quantum properties of the QW, leaving the default values of all parameters
QW_list = QW.GetEffectiveQW(wavelengths=wl)

Solving QW properties...


In [2]:
from solcore import material
from solcore.structure import Layer, Structure
import solcore.quantum_mechanics as QM

T = 300

# First, we create the materials of the QW
QWmat = material('InGaAs')(T=T, In=0.2, strained=True)
Bmat = material('GaAsP')(T=T, P=0.1, strained=True)
i_GaAs = material('GaAs')(T=T)

# The QW is 7 nm wide, with GaAs interlayers 2 nm thick at each side and GaAsP barriers 10 nm thick.
QW = Structure([Layer(width=10e-9, material=Bmat, role="barrier"),
                Layer(width=2e-9, material=i_GaAs, role="interlayer"),
                Layer(width=7e-9, material=QWmat, role="well"),
                Layer(width=2e-9, material=i_GaAs, role="interlayer"),
                Layer(width=10e-9, material=Bmat, role="barrier") ],
               substrate = i_GaAs)

# Finally, the quantum properties are calculated here
output = QM.schrodinger(QW)

In [3]:
from solcore.structure import Junction

T = 300

## Materials for the BOTTOM junction
window_bottom = material('GaInP')(T=T, Nd=5e24, In=0.49)
n_GaAs = material('GaAs')(T=T, Nd=1e24)
p_GaAs = material('GaAs')(T=T, Na=8e22)
bsf_bottom = material('GaInP')(T=T, Na=5e24, In=0.49)

GaAs_junction = Junction([Layer(width=10e-9, material=window_bottom, role="Window"),
                          Layer(width=150e-9, material=n_GaAs, role="Emitter")] +
                         QW_list +
                         [Layer(width=2000e-9, material=p_GaAs, role="Base"),
                          Layer(width=200e-9, material=bsf_bottom, role="BSF")], sn=1e6, sp=1e6, T=T, kind='PDD')

## Materials for the TOP junction
window_top = material('AlInP')(T=T, Nd=5e23, Al=0.53)
n_GaInP = material('GaInP')(T=T, Nd=5e23, In=0.49)
p_GaInP = material('GaInP')(T=T, Na=8e22, In=0.49)
bsf_top = material('AlInP')(T=T, Na=5e24, Al=0.53)

GaInP_junction = Junction([Layer(width=120e-9, material=n_GaInP, role="Emitter"),
                           Layer(width=800e-9, material=p_GaInP, role="Base")], sn=1e3, sp=1e3, T=T, kind='PDD')

In [4]:
from solcore.structure import TunnelJunction

tunnel = TunnelJunction([Layer(width=40e-9, material=n_GaInP, role="TJ")],
                        v_peak=0.2, j_peak=7.5e4, v_valley=1, j_valley=4e4, prefactor=5, j01=1e-23, kind='parametric')

In [5]:
MgF2 = material('MgF2')()
ZnS = material('ZnScub')()

In [7]:
from solcore.solar_cell import SolarCell

my_solar_cell = SolarCell([Layer(width=110e-9, material=MgF2, role="ARC1"),
                           Layer(width=60 - 9, material=ZnS, role="ARC2"),
                           Layer(width=30e-9, material=window_top, role="window"),
                           GaInP_junction,
                           Layer(width=100e-9, material=bsf_top, role="BSF"),
                           tunnel,
                           GaAs_junction],
                          T=T, substrate=n_GaAs)

In [8]:
from solcore.solar_cell_solver import solar_cell_solver
from solcore.light_source import LightSource

light_source = LightSource(source_type='standard', version='AM1.5g', x=wl,
                           output_units='photon_flux_per_m', concentration=1)

solar_cell_solver(my_solar_cell, 'qe',
                  user_options={'light_source': light_source, 'wavelength': wl, 'optics_method': 'TMM'})

C:\Users\mbles\DataspellProjects\EEE_434_functions\venv2\Lib\site-packages\solcore\registries.py:73: UserWarning: Optics solver 'RCWA' will not be available. An installation of S4 has not been found.
  warn(


Treating layer(s) 1 incoherently
Calculating RAT...
Calculating absorption profile...
Solving QE of the solar cell...
Solving quantum efficiency...
Processing structure...
...done!

Solving equilibrium...
...done!

...done!

...done!

Solving quantum efficiency...
Processing structure...
...done!

Solving equilibrium...
...done!

...done!

...done!



In [10]:
my_solar_cell.absorbed

array([7.93167938e-01, 7.94320643e-01, 7.95479904e-01, ...,
       1.37589853e-14, 6.72341862e-15, 6.22309278e-15])

In [14]:
from plotly.express import scatter, line
from pandas import DataFrame

df = {'efficiency':my_solar_cell.absorbed, 'wavelength': wl}

In [17]:
line(df, y= 'efficiency', x = 'wavelength')


In [18]:
scatter(df, y= 'efficiency', x = 'wavelength')